# 물체 검출 모델(SSD) 학습하기
좋은 성능을 달성하기 위해서<br/>
매우 큰 데이터셋(ImageNet 등)에서 영상 분류를 위한 모델을 학습하고,<br/>
학습된 모델의 weights를 가져와서 풀고자 하는 문제에 맞게 학습하는 전이학습(Transfer learning)이 많이 사용됩니다.

다음 코드에서는 이와 비슷하게, 네트워크 대부분은 잘 학습되어 있고 일부만 학습되어있지 않은 경우를 다루게 됩니다.<br/>
즉, Feature extractor의 역할을 하는 VGG16 부분은 제외하고, Classifier의 역할을 하는 layer들만 학습을 진행함으로써,<br/>
짧은 시간동안의 학습으로 성능을 개선하는 실험입니다.

먼저 학습 된 모델 파라미터(weights/SSD300_early_stop.pth)를 VGG16에 해당하는 부분만 로드하고,<br/>
나머지 layer들은 PASCAL VOC2007 에서 학습하여 성능을 평가해보도록 합니다.

학습이 끝나면, practice2.ipynb를 활용하여 성능을 평가해보고 practice1.ipynb를 활용하여 검출 결과를 그려봅시다.

In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from utils import Timer
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np

if torch.cuda.is_available():    
    torch.set_default_tensor_type('torch.cuda.FloatTensor')    
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

## SSD 모델 및 PASCAL VOC2007 데이터 셋 로드
학습에 필요한 hyper-parameter 값들을 정의하고, 모델과 데이터셋을 로드합니다.    

In [2]:
print('Loading the dataset...')

# resume           = None     # specify filename
# resume           = 'weights/ssd300_before_optimize.pth'
resume           = 'weights/ssd300_mAP_77.43_v2.pth'

batch_size       = 4
max_epochs       = 2
lr               = 1e-3
momentum         = 0.9
weight_decay     = 5e-4
lr_schedule      = [int(max_epochs*0.5)]

num_classes      = 21

overlap_thresh   = 0.5    # overlap threshold to determine positive training samples (default boxes)
pos_neg_ratio    = 3      # to alleviate class-imbalance problem (too many negatives), 
                          # limit the negative samples to 3x of positive samples    
    
logging_interval = 100

dataset = VOCDetection(root=VOC_ROOT, transform=SSDAugmentation(300,MEANS))
dataloader = data.DataLoader(dataset, batch_size, num_workers=2, \
                              shuffle=True, collate_fn=detection_collate)
    
net = build_ssd('vgg16', 300, num_classes)    # initialize SSD
cudnn.benchmark = True

if resume:
    print('Resuming training, loading {}...'.format(resume))
    net.load_weights(resume)    # load all weights    
    
else:
    net.apply(init_weights)
        
        
if torch.cuda.is_available():
    net = net.cuda()

criterion = MultiBoxLoss(num_classes, overlap_thresh, \
                         True, 0, True, pos_neg_ratio, 0.5, False, torch.cuda.is_available())

optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_schedule, gamma=0.1)

print( 'Model: {}\n'.format( net.__class__.__name__ ) )
print( net )
print( 'Optimizer: {}\n'.format( optimizer.__class__.__name__ ) )
    

Loading the dataset...
Resuming training, loading weights/ssd300_before_optimize.pth...
Loading weights into state dict...
Finished!
Model: SSD

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), p

## Training loop
아래의 코드는 target과 prediction(network output)으로부터 계산된 loss를 이용하여, <br/>
loss를 줄여가는 방향의 gradient를 구하고 모델을 업데이트 하는 방식으로 모델을 학습하는 코드입니다. <br/>

    1. 데이터 로드 
    2. Forward propagation 
    3. Backward propagation (Gradients 계산)
    4. Loss 계산
    5. Model update
    
    
[TODO] 아래의 각 과정이 코드에서 어느 부분에 대응되는지 생각해보세요.<br/>
[TODO] 아래의 코드에는 심각한 문제가 있습니다. 실행하기 전에 어느 부분이 문제인지 생각해 보세요.<br/>
hint: backward 함수가 실행되면서 network에 있는 각각의 weight가 update되어야하는 gradient가 누.적.해.서. 저장이 됩니다.    

In [3]:
def train(epoch, net, dataloader, cur_lr):
    
    print('Training SSD')
    
    net.train()
    
    # loss counters
    total_loss = 0
    
    sum_loss = 0
    loc_loss = 0
    conf_loss = 0
    
    # timers
    _t = {'forward': Timer(), 'backward': Timer()}    
    
    # load train data
    for batch_idx, (images, targets) in enumerate(dataloader):
        
        if torch.cuda.is_available():
            images = images.cuda()
            with torch.no_grad():
                targets = [ann.cuda() for ann in targets]
                                
        _t['forward'].tic()        
        out = net(images)
        forward_time = _t['forward'].toc(average=True)
                
        _t['backward'].tic()
        
        ### [TODO] 뭔가 중요한 스텝이 생략되었네요!
        optimizer.zero_grad()

        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c   
        loss.backward()
        optimizer.step()
        backward_time = _t['backward'].toc(average=True)
                
        sum_loss += loss.item()
        loc_loss += loss_l.item()
        conf_loss += loss_c.item()
        
        total_loss += loss.item()

        if (batch_idx+1) % logging_interval == 0:            
            print('[Epoch {:3d}][iter {:5d}/{:5d}] Loss: {:7.4f} = {:>7.4f}(loc) + {:>7.4f}(cls) \
                  || forward {:4.2f}s, backward {:4.2f}s || lr: {:.6f}'.format(
                epoch, batch_idx, len(dataloader), \
                sum_loss/logging_interval, loc_loss/logging_interval, conf_loss/logging_interval, \
                forward_time, backward_time, \
                cur_lr
                )
            )   
            
            sum_loss = 0
            loc_loss = 0
            conf_loss = 0

    return total_loss

## 모델 학습 시작
max_epochs 만큼 loop을 돌면서 모델을 학습합니다.
특정 iteration (in train func.) 또는 epoch 마다 learning rate을 조절하는 learning rate scheduling 도 일반적으로 사용됩니다.

In [9]:
for epoch in range(max_epochs):
    optim_scheduler.step()
    train(epoch, net, dataloader, optim_scheduler.get_lr()[0])    
    torch.save(net.state_dict(), 'weights/ssd300_epoch_{:03d}.pth'.format(epoch))
    
print('done.')

Training SSD
[Epoch   0][iter    99/ 4138] Loss:  6.7632 =  1.7997(loc) +  4.9635(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   199/ 4138] Loss:  6.1826 =  1.6623(loc) +  4.5203(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   299/ 4138] Loss:  5.5730 =  1.4713(loc) +  4.1017(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   399/ 4138] Loss:  5.5508 =  1.4552(loc) +  4.0956(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   499/ 4138] Loss:  5.5183 =  1.4700(loc) +  4.0483(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   599/ 4138] Loss:  5.3081 =  1.5337(loc) +  3.7744(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
[Epoch   0][iter   699/ 4138] Loss:  5.1419 =  1.3747(loc) +  3.7672(cls)                   || forward 0.01s, backward 0.04s || lr: 0.001000
